In [1]:
# Bring your packages onto the path

from crawl_stock_data.service import process_data_service
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from pandas.plotting import register_matplotlib_converters
import warnings

warnings.filterwarnings('ignore')
register_matplotlib_converters()
%matplotlib inline

In [56]:
#get the dataset from the csv file into df

stock_name = 'GOOGL'
df = process_data(stock_name)
df.head(3)

,Date,open,high,low,close,volume,adj_close,NATR_3,RSI_3,ADX_3,...,BBM_3_2.0,BBU_3_2.0,BBB_3_2.0,BBP_3_2.0,min_price_3,max_price_3,mid_price,tema_2,tema_8,tema_100
0,2005-01-10,4.867367,4.957457,4.800551,4.881381,301282416.0,4.881381,3.297238,58.767641,42.870467,...,4.816983,4.958482,5.874972,0.727556,4.697698,4.957457,4.879004,4.883820,4.850060,3.863238
1,2005-01-11,4.895395,4.947698,4.834334,4.843343,278069652.0,4.843343,2.831877,48.714336,37.850269,...,4.858608,4.891431,1.351113,0.267455,4.724224,4.957457,4.891016,4.848029,4.842952,3.920317
2,2005-01-12,4.863113,4.903153,4.767267,4.889389,326784888.0,4.889389,2.792202,60.869291,34.702612,...,4.871371,4.911544,1.649371,0.724253,4.767267,4.957457,4.835210,4.887231,4.862860,3.977832


In [57]:
#rename the column in df
df = df.rename(columns={'Volume': 'volume',
                        'Open': 'open', 'High': 'high',
                        'Low': 'low', 'Close': 'close', 'Date': 'date'})
df.tail(2)

,date,open,high,low,close,volume,adj_close,NATR_3,RSI_3,ADX_3,...,BBM_3_2.0,BBU_3_2.0,BBB_3_2.0,BBP_3_2.0,min_price_3,max_price_3,mid_price,tema_2,tema_8,tema_100
4670,2023-08-01,130.779999,132.630005,130.679993,131.550003,23166800.0,131.550003,2.002916,70.580813,60.694002,...,132.283335,133.326704,1.577476,0.148575,130.570007,133.740005,131.654999,131.604447,133.172458,129.677020
4671,2023-08-02,129.449997,130.089996,127.559998,128.380005,26244300.0,128.380005,2.580169,34.747437,53.121827,...,130.883336,134.550206,5.603265,0.158656,127.559998,133.529999,128.824997,128.425960,131.450224,129.966347


In [58]:
#Change the type of the date column from object to datetime in df

df['date'] = pd.to_datetime(df['date'], utc=True)
df["date"] = df["date"].dt.strftime("%Y-%m-%d")
df.index = pd.DatetimeIndex(df["date"])  #assign the changed column to the index
df.sort_index(inplace=True)
df = df.loc['1981-05-06':]
print(len(df))
df.last('3D')

4672


,date,open,high,low,close,volume,adj_close,NATR_3,RSI_3,ADX_3,...,BBM_3_2.0,BBU_3_2.0,BBB_3_2.0,BBP_3_2.0,min_price_3,max_price_3,mid_price,tema_2,tema_8,tema_100
date,,,,,,,,,,,,,,,,,,,,,
2023-07-31,2023-07-31,132.729996,133.529999,131.779999,132.720001,28055500.0,132.720001,2.433442,94.580197,73.628372,...,131.566666,134.632934,4.661162,0.688068,128.789993,133.740005,132.654999,132.818988,132.926914,129.178233
2023-08-01,2023-08-01,130.779999,132.630005,130.679993,131.550003,23166800.0,131.550003,2.002916,70.580813,60.694002,...,132.283335,133.326704,1.577476,0.148575,130.570007,133.740005,131.654999,131.604447,133.172458,129.677020
2023-08-02,2023-08-02,129.449997,130.089996,127.559998,128.380005,26244300.0,128.380005,2.580169,34.747437,53.121827,...,130.883336,134.550206,5.603265,0.158656,127.559998,133.529999,128.824997,128.425960,131.450224,129.966347


In [59]:
#drop column date if it exists in df

if 'date' in df.columns:
    df.drop(['date'], axis=1, inplace=True)

#check for null values in df
df.isnull().sum()

open             0
high             0
low              0
close            0
volume           0
adj_close        0
NATR_3           0
RSI_3            0
ADX_3            0
CCI_3_0.015      0
ROC_3            0
STOCHk_14_3_3    0
STOCHd_14_3_3    0
WILLR_3          0
OBV              0
MACD_12_26_9     0
BBL_3_2.0        0
BBM_3_2.0        0
BBU_3_2.0        0
BBB_3_2.0        0
BBP_3_2.0        0
min_price_3      0
max_price_3      0
mid_price        0
tema_2           0
tema_8           0
tema_100         0
dtype: int64

In [60]:
df['open'] = df['open'].astype(float)
df['high'] = df['high'].astype(float)
df['low'] = df['low'].astype(float)
df['close'] = df['close'].astype(float)
df['volume'] = df['volume'].astype(float)
###get the dataset of apple stock from 2013-01-01 to 2023-06-01 from df
df = df.loc['2013-01-01':]
df.index

DatetimeIndex(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07',
               '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11',
               '2013-01-14', '2013-01-15',
               ...
               '2023-07-20', '2023-07-21', '2023-07-24', '2023-07-25',
               '2023-07-26', '2023-07-27', '2023-07-28', '2023-07-31',
               '2023-08-01', '2023-08-02'],
              dtype='datetime64[ns]', name='date', length=2664, freq=None)

In [61]:
#We need to restructure the data set by adding a target column ‘Actual’, which will the stock prices for the next day.

steps = -1
main_dataset = df.copy()
main_dataset['Next date'] = main_dataset['close'].shift(steps)
main_dataset.last('3D')
main_dataset.isnull().sum()

open             0
high             0
low              0
close            0
volume           0
adj_close        0
NATR_3           0
RSI_3            0
ADX_3            0
CCI_3_0.015      0
ROC_3            0
STOCHk_14_3_3    0
STOCHd_14_3_3    0
WILLR_3          0
OBV              0
MACD_12_26_9     0
BBL_3_2.0        0
BBM_3_2.0        0
BBU_3_2.0        0
BBB_3_2.0        0
BBP_3_2.0        0
min_price_3      0
max_price_3      0
mid_price        0
tema_2           0
tema_8           0
tema_100         0
Next date        1
dtype: int64

In [62]:
##dropping columns with null values in dataset_for_prediction

main_dataset.dropna(inplace=True)
main_dataset.last('3D')

,open,high,low,close,volume,adj_close,NATR_3,RSI_3,ADX_3,CCI_3_0.015,...,BBU_3_2.0,BBB_3_2.0,BBP_3_2.0,min_price_3,max_price_3,mid_price,tema_2,tema_8,tema_100,Next date
date,,,,,,,,,,,,,,,,,,,,,
2023-07-31,132.729996,133.529999,131.779999,132.720001,28055500.0,132.720001,2.433442,94.580197,73.628372,64.178874,...,134.632934,4.661162,0.688068,128.789993,133.740005,132.654999,132.818988,132.926914,129.178233,131.550003
2023-08-01,130.779999,132.630005,130.679993,131.550003,23166800.0,131.550003,2.002916,70.580813,60.694002,-100.000000,...,133.326704,1.577476,0.148575,130.570007,133.740005,131.654999,131.604447,133.172458,129.677020,128.380005


In [63]:
#splitting the data into training and test sets

train_df = main_dataset[:'2022-12-30']
test_df = main_dataset['2023-01-01':]

In [64]:
train_X = train_df[['low', 'high', 'open', 'close', 'volume',
                    'NATR_3', 'RSI_3', 'ADX_3', 'CCI_3_0.015',
                    'ROC_3', 'STOCHk_14_3_3', 'STOCHd_14_3_3',
                    'WILLR_3', 'OBV', 'MACD_12_26_9', 'BBL_3_2.0',
                    'BBM_3_2.0', 'BBU_3_2.0', 'BBB_3_2.0',
                    'BBP_3_2.0', 'min_price_3', 'max_price_3',
                    'mid_price', 'tema_2', 'tema_100', 'tema_8']]

train_y = train_df[['Next date']]
train_X.head(10)

,low,high,open,close,volume,NATR_3,RSI_3,ADX_3,CCI_3_0.015,ROC_3,...,BBM_3_2.0,BBU_3_2.0,BBB_3_2.0,BBP_3_2.0,min_price_3,max_price_3,mid_price,tema_2,tema_100,tema_8
date,,,,,,,,,,,,,,,,,,,,,
2023-07-19,121.800003,125.180000,124.599998,122.029999,37224000.0,2.427984,36.933985,35.055744,-81.201063,-2.702918,...,123.480001,125.655565,3.523751,0.166753,121.800003,127.099998,123.490002,122.023657,127.092154,123.847177
2023-07-20,118.220001,124.089996,121.419998,119.199997,37906800.0,3.705061,19.281771,42.694503,-100.000000,-4.372246,...,121.663333,125.422497,6.179618,0.172356,118.220001,125.180000,121.154999,119.206784,126.993906,121.669975
2023-07-21,118.730003,120.989998,120.620003,120.019997,72937900.0,2.781381,33.165125,47.787009,-65.831801,-3.021982,...,120.416664,122.794471,3.949298,0.416590,118.220001,125.180000,119.860001,119.853145,126.942148,120.759212
2023-07-24,120.980003,123.000000,121.660004,121.529999,29686100.0,2.599447,54.691052,34.207433,100.000000,-0.409735,...,120.249997,122.180042,3.210054,0.831599,118.220001,124.089996,121.990002,121.470280,126.975314,121.019978
2023-07-25,121.019997,123.150002,121.360001,122.209999,52509600.0,2.163945,62.787130,26.519069,60.515651,2.525170,...,121.253332,123.083768,3.019193,0.761322,118.730003,123.150002,122.084999,122.262445,127.041794,121.555730
2023-07-26,128.320007,130.979996,130.070007,129.270004,61682100.0,4.415005,90.162544,44.354743,100.000000,7.707055,...,124.336667,131.335517,11.257901,0.852439,120.980003,130.979996,129.650002,129.069140,127.511877,125.624413
2023-07-27,128.789993,133.240005,131.669998,129.399994,44952100.0,3.924764,90.358433,57.918274,59.081239,6.475763,...,126.959999,133.678352,10.583417,0.681592,121.019997,133.240005,131.014999,129.672399,127.966136,128.103488
2023-07-28,130.570007,133.740005,130.779999,132.580002,36591200.0,3.552065,94.429088,67.344333,100.000000,8.485396,...,130.416667,133.477925,4.694582,0.853341,128.320007,133.740005,132.155006,132.551674,128.582751,131.167374
2023-07-31,131.779999,133.529999,132.729996,132.720001,28055500.0,2.433442,94.580197,73.628372,64.178874,2.668830,...,131.566666,134.632934,4.661162,0.688068,128.789993,133.740005,132.654999,132.818988,129.178233,132.926914


In [65]:
test_X = test_df[['low', 'high', 'open', 'close', 'volume',
                  'NATR_3', 'RSI_3', 'ADX_3', 'CCI_3_0.015',
                  'ROC_3', 'STOCHk_14_3_3', 'STOCHd_14_3_3',
                  'WILLR_3', 'OBV', 'MACD_12_26_9', 'BBL_3_2.0',
                  'BBM_3_2.0', 'BBU_3_2.0', 'BBB_3_2.0',
                  'BBP_3_2.0', 'min_price_3', 'max_price_3',
                  'mid_price', 'tema_2', 'tema_100', 'tema_8']]

test_Y = test_df[['Next date']]
test_X.head(10)

,Next date
date,
2023-07-19,119.199997
2023-07-20,120.019997
2023-07-21,121.529999
2023-07-24,122.209999
2023-07-25,129.270004
2023-07-26,129.399994
2023-07-27,132.580002
2023-07-28,132.720001
2023-07-31,131.550003


In [66]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler_training_set = StandardScaler()
scaler_training_set_fit = scaler_training_set.fit(train_X)
scaler_training_set = scaler_training_set_fit.transform(train_X)
scaler_training_set = pd.DataFrame(scaler_training_set)
train_X = scaler_training_set

# scaler_test_set = StandardScaler()
# scaler_test_set_fit = scaler_test_set.fit(test_X)
scaler_test_set = scaler_training_set_fit.transform(test_X)
scaler_test_set = pd.DataFrame(scaler_test_set)
test_X = scaler_test_set

train_X.tail(10)

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
2653,1.717035,1.733471,1.757051,1.683713,-0.198835,0.265369,-0.705060,-0.945324,-1.095171,-1.028863,...,1.726984,1.715149,0.005338,-1.340362,1.759398,1.751877,1.725459,1.683526,1.826766,1.734766
2654,1.615093,1.703183,1.667587,1.604119,-0.173063,1.453948,-1.399267,-0.499434,-1.326818,-1.606816,...,1.675853,1.708738,0.927250,-1.319399,1.656107,1.699108,1.659778,1.604302,1.824006,1.673536
2655,1.629616,1.617043,1.645081,1.627182,1.149193,0.594277,-0.853278,-0.202174,-0.905785,-1.139329,...,1.640764,1.636450,0.153055,-0.405718,1.656107,1.699108,1.623352,1.622481,1.822552,1.647922
2656,1.693686,1.672895,1.674340,1.669650,-0.483355,0.424951,-0.006729,-0.994842,1.137654,-0.234918,...,1.636073,1.619549,-0.103553,1.146838,1.656107,1.669150,1.683266,1.667962,1.823483,1.655256
2657,1.694824,1.677063,1.665900,1.688775,0.378123,0.019627,0.311664,-1.443627,0.651114,0.781203,...,1.664313,1.644407,-0.169805,0.883932,1.670822,1.643316,1.685938,1.690242,1.825351,1.670323
2658,1.902696,1.894635,1.910940,1.887337,0.724341,2.114695,1.388255,-0.402522,1.137654,2.575272,...,1.751096,1.871385,2.690037,1.224802,1.735739,1.858514,1.898732,1.881679,1.838557,1.784748
2659,1.916079,1.957434,1.955953,1.890993,0.092864,1.658427,1.395959,0.389208,0.633438,2.148975,...,1.824932,1.935828,2.455908,0.585659,1.736893,1.920628,1.937128,1.898645,1.851318,1.854467
2660,1.966765,1.971327,1.930914,1.980431,-0.222720,1.311555,1.556045,0.939427,1.137654,2.844749,...,1.922223,1.930315,0.411760,1.228175,1.947514,1.934370,1.969195,1.979624,1.868641,1.940634
2661,2.001221,1.965492,1.985774,1.984368,-0.544902,0.270449,1.561988,1.306239,0.696253,0.830941,...,1.954590,1.962086,0.400159,0.609888,1.961074,1.934370,1.983259,1.987142,1.885369,1.990118
2662,1.969897,1.940484,1.930914,1.951462,-0.729427,-0.130243,0.618166,0.551233,-1.326818,0.482191,...,1.974762,1.926156,-0.670258,-1.408365,2.012431,1.934370,1.955130,1.952984,1.899382,1.997023


In [67]:
#Scaling the target feature
scaler_output_training_set = StandardScaler()
scaler_output_training_set_fit = scaler_output_training_set.fit(train_y)
scaler_output_training_set = scaler_output_training_set_fit.transform(train_y)
scaler_output_training_set = pd.DataFrame(scaler_output_training_set)
train_y = scaler_output_training_set

# scaler_output_test_set =  StandardScaler()
# scaler_output_test_set_fit = scaler_output_test_set.fit(test_Y)
scaler_output_test_set = scaler_output_training_set_fit.transform(test_Y)
scaler_output_test_set = pd.DataFrame(scaler_output_test_set)
test_Y = scaler_output_test_set

train_y.tail(10)

,Next date
date,
2023-07-19,119.199997
2023-07-20,120.019997
2023-07-21,121.529999
2023-07-24,122.209999
2023-07-25,129.270004
2023-07-26,129.399994
2023-07-27,132.580002
2023-07-28,132.720001
2023-07-31,131.550003


In [68]:
#rename X

train_X.rename(columns={0: 'open', 1: 'high', 2: 'low', 3: 'close', 4: 'volume', 5: 'NATR_3', 6: 'RSI_3', 7: 'ADX_3',
                        8: 'CCI_3_.015',
                        9: 'ROC_3', 10: 'STOCHk_14_3_3', 11: 'STOCHd_14_3_3', 12: 'WILLR_3', 13: 'OBV',
                        14: 'MACD_12_26_9',
                        15: 'BBL_3_2.0', 16: 'BBM_3_2.0', 17: 'BBU_3_2.0', 18: 'BBB_3_2.0', 19: 'BBP_3_2.0',
                        20: 'min_price_3',
                        21: 'max_price_3', 22: 'mid_price', 23: 'tema_2', 24: 'tema_100', 25: 'tema_8'}, inplace=True)

test_X.rename(columns={0: 'open', 1: 'high', 2: 'low', 3: 'close', 4: 'volume', 5: 'NATR_3', 6: 'RSI_3', 7: 'ADX_3',
                       8: 'CCI_3_.015',
                       9: 'ROC_3', 10: 'STOCHk_14_3_3', 11: 'STOCHd_14_3_3', 12: 'WILLR_3', 13: 'OBV',
                       14: 'MACD_12_26_9',
                       15: 'BBL_3_2.0', 16: 'BBM_3_2.0', 17: 'BBU_3_2.0', 18: 'BBB_3_2.0', 19: 'BBP_3_2.0',
                       20: 'min_price_3',
                       21: 'max_price_3', 22: 'mid_price', 23: 'tema_2', 24: 'tema_100', 25: 'tema_8'}, inplace=True)

train_X.index = train_df.index
test_X.index = test_df.index

train_X.head(3)

,open,high,low,close,volume,NATR_3,RSI_3,ADX_3,CCI_3_0.015,ROC_3,...,BBM_3_2.0,BBU_3_2.0,BBB_3_2.0,BBP_3_2.0,min_price_3,max_price_3,mid_price,tema_2,tema_100,tema_8
date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,-1.240664,-1.239367,-1.241850,-1.239328,2.229175,0.009918,1.011809,0.467495,1.137654,0.738307,...,-1.248224,-1.238980,0.677936,1.163690,-1.252262,-1.241300,-1.240084,-1.239576,-1.243984,-1.246333
2013-01-03,-1.237693,-1.235939,-1.237970,-1.239033,1.892674,-0.271951,1.029548,0.516478,0.674255,1.077142,...,-1.242669,-1.236462,0.248213,0.593383,-1.252262,-1.237910,-1.236881,-1.238536,-1.243120,-1.242333
2013-01-04,-1.232733,-1.229305,-1.234866,-1.228968,2.564319,-0.027405,1.434178,0.857572,1.137654,1.404136,...,-1.235487,-1.230462,0.086633,1.228555,-1.237424,-1.231348,-1.231074,-1.229165,-1.241711,-1.234678


In [69]:
#rename Y

train_y.rename(columns={0: 'Next date'}, inplace=True)
test_Y.rename(columns={0: 'Next date'}, inplace=True)

train_y.index = train_df.index
test_Y.index = test_df.index

train_y.head(3)

,Next date
date,
2013-01-02,18.109859
2013-01-03,18.467718
2013-01-04,18.387136


In [70]:
train_X.columns

Index(['open', 'high', 'low', 'close', 'volume', 'NATR_3', 'RSI_3', 'ADX_3',
       'CCI_3_0.015', 'ROC_3', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'WILLR_3',
       'OBV', 'MACD_12_26_9', 'BBL_3_2.0', 'BBM_3_2.0', 'BBU_3_2.0',
       'BBB_3_2.0', 'BBP_3_2.0', 'min_price_3', 'max_price_3', 'mid_price',
       'tema_2', 'tema_100', 'tema_8'],
      dtype='object')

In [71]:
#Get only tema_8 from train_X, test_X

train_X_with_tema_8 = train_X.drop(['tema_2', 'tema_100'], axis=1)
test_X_with_tema_8 = test_X.drop(['tema_2', 'tema_100'], axis=1)

train_X_with_tema_8.columns

Index(['open', 'high', 'low', 'close', 'volume', 'NATR_3', 'RSI_3', 'ADX_3',
       'CCI_3_0.015', 'ROC_3', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'WILLR_3',
       'OBV', 'MACD_12_26_9', 'BBL_3_2.0', 'BBM_3_2.0', 'BBU_3_2.0',
       'BBB_3_2.0', 'BBP_3_2.0', 'min_price_3', 'max_price_3', 'mid_price',
       'tema_8'],
      dtype='object')

In [72]:
from sklearn.metrics import mean_squared_error


def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [73]:
test_X_with_tema_8

In [74]:
from sklearn.exceptions import ConvergenceWarning,DataConversionWarning

# Filter out DataConversionWarning
warnings.filterwarnings("ignore", category=DataConversionWarning)

# Filter out ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [75]:
from sklearn.svm import SVR

svr =SVR(C=1, kernel= 'linear', max_iter= 5000)

In [76]:
#Fit the model

svm_model = svr.fit(train_X_with_tema_8, train_y)
forecast = svm_model.predict(test_X_with_tema_8)

In [77]:

#Plotting the forecasted values
forecast_apple_with_tema_2 = pd.DataFrame(forecast)
forecast_apple_with_tema_2.rename(columns={0: 'predicted_mean'}, inplace=True)
forecast_apple_with_tema_2.reset_index(drop=True, inplace=True)
forecast_apple_with_tema_2.index = test_Y.index
forecast_apple_with_tema_2["Actual"] = test_Y
forecast_apple_with_tema_2.head(10)

In [78]:
# # #reverse scale the data to the original form
# 
forecast_apple_with_tema_2["predicted_mean"] = (scaler_output_training_set_fit.inverse_transform
                                                (forecast_apple_with_tema_2[["predicted_mean"]]))
forecast_apple_with_tema_2["Actual"] = (scaler_output_training_set_fit.inverse_transform
                                        (forecast_apple_with_tema_2[["Actual"]]))

In [79]:
forecast_apple_with_tema_2["predicted_mean"].plot(legend=True)
forecast_apple_with_tema_2["Actual"].plot(legend=True)

In [80]:

from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import mean_absolute_percentage_error

error = calculate_rmse(forecast_apple_with_tema_2["predicted_mean"], forecast_apple_with_tema_2["Actual"])
print("The root mean squared error is {}.".format(error))

##calculate the mean absolute percentage error

mape = calculate_mape(forecast_apple_with_tema_2["Actual"],
                      forecast_apple_with_tema_2["predicted_mean"])
print("The mean absolute percentage error is {}.".format(mape))

['/mnt/learning/last_project/app/server/stock_server/crawl_stock_data/training/checkpoint/svm/svr_google_tema_8_13-23.pkl']

In [ ]:
joblib.dump(svr, '/mnt/learning/last_project/fetch_data_stock/trainning_model/svr_google_tema_8.pkl')
